## Introduction

## Set up Python Libraries

In [39]:
#install some Python packages with pip

!pip install optuna nltk numpy torch datasets transformers requests beautifulsoup4 pandas evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.6 MB/s eta 0:00:00


In [2]:
# let's check the version we are using

!pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

datasets==4.0.0
evaluate==0.4.6
numpy==2.0.2
torch==2.9.0+cu126
torchao==0.10.0
torchaudio==2.9.0+cu126
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.24.0+cu126
transformers==4.57.2


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup ## for scraping
import re

In [4]:
import numpy as np
import pandas as pd
from datasets import Dataset

## Instantiate Model

In [6]:
# let's import the pretrained faster tokenizer from huggingface
# source: (https://huggingface.co/distilbert-base-uncased)

checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
tokenizer

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

## Scrape for Trust Pilot Reviews

Scrape first 3 pages of the Expedia, TripAdvisor, Booking.com, Airline Ticket Centre and Gala Travels pages on ca.TrustPilot.

In [9]:
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

In [10]:
reviews = []
ratings = []

for i in range(1, 4): # Loop through pages 1 to 3
    url = f'https://ca.trustpilot.com/review/www.expedia.com?page={i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser') # Use r.text and html.parser for robust parsing

    # Find review headers with class containing 'styles_reviewHeader'
    # and extract 'data-service-review-rating' attribute
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewHeader.*')}), ratings, attr='data-service-review-rating')

    # Find review content divs with class containing 'styles_reviewContent'
    # and extract their text content
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewContent.*')}), reviews)


In [11]:
for i in range(1, 4): # Loop through pages 1 to 3
    url = f'https://ca.trustpilot.com/review/www.tripadvisor.ca?page={i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser') # Use r.text and html.parser for robust parsing

    # Find review headers with class containing 'styles_reviewHeader'
    # and extract 'data-service-review-rating' attribute
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewHeader.*')}), ratings, attr='data-service-review-rating')

    # Find review content divs with class containing 'styles_reviewContent'
    # and extract their text content
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewContent.*')}), reviews)


In [12]:
for i in range(1, 4): # Loop through pages 1 to 3
    url = f'https://ca.trustpilot.com/review/www.booking.com?page={i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser') # Use r.text and html.parser for robust parsing

    # Find review headers with class containing 'styles_reviewHeader'
    # and extract 'data-service-review-rating' attribute
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewHeader.*')}), ratings, attr='data-service-review-rating')

    # Find review content divs with class containing 'styles_reviewContent'
    # and extract their text content
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewContent.*')}), reviews)


In [13]:
for i in range(1, 4): # Loop through pages 1 to 3
    url = f'https://ca.trustpilot.com/review/www.airlineticketcentre.ca?page={i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser') # Use r.text and html.parser for robust parsing

    # Find review headers with class containing 'styles_reviewHeader'
    # and extract 'data-service-review-rating' attribute
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewHeader.*')}), ratings, attr='data-service-review-rating')

    # Find review content divs with class containing 'styles_reviewContent'
    # and extract their text content
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewContent.*')}), reviews)


In [14]:
for i in range(1, 4): # Loop through pages 1 to 3
    url = f'https://ca.trustpilot.com/review/galatravels.com?page={i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser') # Use r.text and html.parser for robust parsing

    # Find review headers with class containing 'styles_reviewHeader'
    # and extract 'data-service-review-rating' attribute
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewHeader.*')}), ratings, attr='data-service-review-rating')

    # Find review content divs with class containing 'styles_reviewContent'
    # and extract their text content
    soup2list(soup.find_all('div', {'class': re.compile(r'.*styles_reviewContent.*')}), reviews)


review_data = pd.DataFrame(
{
   'text':reviews,
   'label': ratings
})


In [15]:
review_data

,text,label
0,WORST SERVICE EVERWORST SERVICE EVER. Hidden f...,1
1,Wow should have read the reviews before…Wow sh...,1
2,I booked a package that included stay…I booked...,1
3,Super badSuper bad. Sometimes agent told me to...,1
4,Expensive Holiday for meToo expensive but I ha...,1
...,...,...
295,Great customer service from AazminGreat custom...,5
296,I would like to state that we know only Jaini ...,5
297,Thank you so much Gala TravelsThank you so muc...,5
298,There is always an agent a call away…There is ...,4


We need to minus 1 from all ratings for training later.

In [16]:
# Convert the 'label' column to numeric type (it's currently a string from scraping)
review_data['label'] = pd.to_numeric(review_data['label'])

# Subtract 1 from the 'label' column
review_data['label'] = review_data['label'] - 1

# Display the updated DataFrame head to confirm the change
print(review_data.head())

                                                text  label
0  WORST SERVICE EVERWORST SERVICE EVER. Hidden f...      0
1  Wow should have read the reviews before…Wow sh...      0
2  I booked a package that included stay…I booked...      0
3  Super badSuper bad. Sometimes agent told me to...      0
4  Expensive Holiday for meToo expensive but I ha...      0


## Load Reviews into DataFrame and Make csv

In [17]:
# Save the raw reviews to a CSV file
import pandas as pd
review_data.to_csv('trustpilot_reviews.csv', index=False)
print(f"DataFrame with {len(review_data)} entries (text and label) saved to 'trustpilot_reviews.csv'")

DataFrame with 300 entries (text and label) saved to 'trustpilot_reviews.csv'


## Tokenize and Split Dataset

Make function for tokenizing dataset.

In [18]:
from transformers import BertTokenizer

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # Ensures uniform input size
        max_length=128  # Adjust based on task
    )

Split csv dataset into dictionary for train, test & validate.

In [19]:
df = pd.read_csv('trustpilot_reviews.csv')
# Initial dataset before tokenization, named 'raw_dataset' to avoid confusion with the final 'dataset' DatasetDict
raw_dataset = Dataset.from_pandas(df)

# Tokenize the raw dataset
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)

# Rename the 'label' column to 'labels' as expected by the Trainer
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# Split the tokenized_dataset into training (80%) and a temporary set (20%)
train_test_valid_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Split the temporary set (train_test_valid_split['test']) into validation (50% of temp) and test (50% of temp)
test_valid_split = train_test_valid_split['test'].train_test_split(test_size=0.5, seed=42)

# Create a DatasetDict named 'dataset' as requested by the user
dataset = {
    'train': train_test_valid_split['train'],
    'val': test_valid_split['train'],
    'test': test_valid_split['test']
}

# Now 'dataset' is a DatasetDict and can be accessed as dataset["train"][0], etc.
print(f"Dataset split into: {len(dataset['train'])} training samples, {len(dataset['val'])} validation samples, {len(dataset['test'])} test samples.")
print(dataset["train"][0]) # Example access as requested by the user

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset split into: 240 training samples, 30 validation samples, 30 test samples.
{'text': 'Serious data-sharing and brand-damaging by Expedia that every hotelier should be aware ofI am posting this review to warn other hoteliers about practices that can seriously damage your brand without your knowledge.We recently discovered that Expedia Private Label Solutions was sharing our property listings, photos, data and even room descriptions with unverified third-party websites that we had never heard of and never authorized.Two of these websites were:• VaudHotels – which displays photos of rooms that do not even exist in our property, cannot be contacted, and has no legal information available anywhere online• HotelsOne.com – whose management confirmed in writing that they receive our listings directly from Expedia Private Label SolutionsBoth of these websites have already been reported to the Swiss authorities, and I am prepared to do the same with Expedia if this continues.The most conce

Sample tokenized output:

In [34]:
print(dataset["train"][0])

{'text': 'Serious data-sharing and brand-damaging by Expedia that every hotelier should be aware ofI am posting this review to warn other hoteliers about practices that can seriously damage your brand without your knowledge.We recently discovered that Expedia Private Label Solutions was sharing our property listings, photos, data and even room descriptions with unverified third-party websites that we had never heard of and never authorized.Two of these websites were:• VaudHotels – which displays photos of rooms that do not even exist in our property, cannot be contacted, and has no legal information available anywhere online• HotelsOne.com – whose management confirmed in writing that they receive our listings directly from Expedia Private Label SolutionsBoth of these websites have already been reported to the Swiss authorities, and I am prepared to do the same with Expedia if this continues.The most concerning part is that when I asked Expedia for the full list of OTA partners, affilia

## Evaluating the Model

In [21]:
import evaluate
import numpy as np

# we setup the training to evaluate the accuracy and f1 scores

accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    # Changed 'average' from default 'binary' to 'weighted' for multiclass classification
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {**accuracy, **f1}

In [ ]:
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy

# get bert model with a sequence classification head for sentiment analysis
# source: (https://huggingface.co/distilbert-base-uncased)
checkpoint = 'distilbert-base-uncased'
num_labels = 5
id2label = {0:'1 star', 1:'2 stars', 2:'3 stars', 3:'4 stars', 4:'5 stars'}
label2id = {'1 star':0, '2 stars':1, '3 stars':2, '4 stars':3, '5 stars':4}
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training
training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch'
)

# setup trainer with custom metrics (accuracy, f1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    compute_metrics=compute_metrics,
)

# disable wandb logging (a v4 huggingface artifact)
os.environ['WANDB_DISABLED']= "true"

Test for unfine-tuned model.

In [23]:
trainer.evaluate(dataset['test'])

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


wandb: Currently logged in as: artpurposesdennys (artpurposesdennys-simon-fraser-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 1.7093764543533325,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.03333333333333333,
 'eval_f1': 0.014814814814814814,
 'eval_runtime': 5.115,
 'eval_samples_per_second': 5.865,
 'eval_steps_per_second': 0.391}

In [24]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
1,No log,0.885129,0.001600,0.900000,0.868867


TrainOutput(global_step=15, training_loss=1.2309365590413412, metrics={'train_runtime': 205.7028, 'train_samples_per_second': 1.167, 'train_steps_per_second': 0.073, 'total_flos': 7948469145600.0, 'train_loss': 1.2309365590413412, 'epoch': 1.0})

In [25]:
trainer.evaluate(dataset['test']) ## evaluate on test set

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.9470030665397644,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.8,
 'eval_f1': 0.7771428571428572,
 'eval_runtime': 7.5639,
 'eval_samples_per_second': 3.966,
 'eval_steps_per_second': 0.264,
 'epoch': 1.0}

## Testing with examples

In [28]:
from transformers import pipeline
import torch

# create pipeline for sentiment classifier with custom model and tokenizer
sentiment_classifier = pipeline(task='sentiment-analysis', model=model, tokenizer=tokenizer)

Device set to use cpu


In [29]:
# let's see how our model classifies a bad review (1 star)
# this is from (https://ca.trustpilot.com/review/www.expedia.ca?page=5)

review = """
Hote bill error, expedia.ca fault
I used to like Expedia. Now I will certainly reconsider further bookings with them.
I have booked a hotel near Cancun and the invoice from Expedia included a tax to be charged by the hotel.
This is normal, there are various and numerous "environmental" taxes these days. However, the hotel actually
requested the amount that was 3x more than what Expedia.ca provided in the invoice. As far as I understand,
it was Expedia's fault, they calculated it incorrectly. However, Expedia has rejected my claim and even the proposal
to compensate me the difference in Expedia points! I guess, this is the warning sign for me - with Expedia,
the customer is always wrong.
"""
sentiment_classifier(review)

[{'label': '1 star', 'score': 0.46419504284858704}]

In [ ]:
# let's see how our model classifies a 5 star review
# this is from (https://ca.trustpilot.com/review/www.airlineticketcentre.ca)

review = """
A Grateful Customer's Appreciation for Airline Ticket
I’ve been a loyal client of Airline Ticket for over 20 years,
and their service has consistently been outstanding.
They are incredibly reliable—whether it's booking, inquiries,
or last-minute changes, they always deliver with professionalism and care.

One of the things I value most is their commitment to personal service.
They always answer their landline promptly, and it’s always a real human on
the other end—ready to help, not just route you through automated systems.
Their customer service is truly exceptional: responsive, knowledgeable, and
 genuinely dedicated to guiding and supporting customers at any time.

I’m especially thankful to Michal, Judy, and most recently Shahir
for their continued professionalism and kindness. Their expertise and
personal touch make every interaction smooth and reassuring.

Thank you for two decades of excellence!
"""
sentiment_classifier(review)

## Fine-tuning parameters

Finding the best parameters using Optuna.It uses a smarter approach to search the hyperparameter space and focuses on promising regions. I’ve used this in several projects, and it often finds better configurations than manual tuning.

In [48]:
import optuna
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }


def objective(trial):
    # Hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])

    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=1,
        weight_decay=0.01,
        eval_strategy="epoch",
        logging_dir="./logs",
        report_to="none",         # avoid wandb warnings
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"]


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best params:", study.best_params)


[I 2025-12-01 01:11:18,969] A new study created in memory with name: no-name-5c8e211e-3bcd-4a91-8b55-97e97e8457e9


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.312048,0.900000,0.871685


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[I 2025-12-01 01:14:28,568] Trial 0 finished with value: 0.9 and parameters: {'learning_rate': 2.5542566639714656e-05, 'batch_size': 16}. Best is trial 0 with value: 0.9.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.323822,0.900000,0.871685


...

[I 2025-12-01 02:08:08,627] Trial 18 finished with value: 0.9 and parameters: {'learning_rate': 1.4170443969412987e-05, 'batch_size': 8}. Best is trial 2 with value: 0.9333333333333333.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.651222,0.866667,0.890622


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[I 2025-12-01 02:11:06,158] Trial 19 finished with value: 0.8666666666666667 and parameters: {'learning_rate': 4.801200214133162e-05, 'batch_size': 16}. Best is trial 2 with value: 0.9333333333333333.


Best params: {'learning_rate': 2.734763905921527e-05, 'batch_size': 8}


Put best parameters in training

In [65]:
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy

# get bert model with a sequence classification head for sentiment analysis
# source: (https://huggingface.co/distilbert-base-uncased)
checkpoint = 'distilbert-base-uncased'
num_labels = 5
id2label = {0:'1 star', 1:'2 stars', 2:'3 stars', 3:'4 stars', 4:'5 stars'}
label2id = {'1 star':0, '2 stars':1, '3 stars':2, '4 stars':3, '5 stars':4}
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training
training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,

    # Best parameters from Optuna
    learning_rate=2.734763905921527e-05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,

    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch'
)

# setup trainer with custom metrics (accuracy, f1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    compute_metrics=compute_metrics,
)

# disable wandb logging (a v4 huggingface artifact)
os.environ['WANDB_DISABLED']= "true"

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [66]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.833147,0.900000,0.868867


TrainOutput(global_step=30, training_loss=1.180194091796875, metrics={'train_runtime': 274.7971, 'train_samples_per_second': 0.873, 'train_steps_per_second': 0.109, 'total_flos': 7948469145600.0, 'train_loss': 1.180194091796875, 'epoch': 1.0})

In [67]:
trainer.evaluate(dataset['test']) ## evaluate on test set

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.8935801982879639,
 'eval_accuracy': 0.8666666666666667,
 'eval_f1': 0.8406349206349206,
 'eval_runtime': 8.4329,
 'eval_samples_per_second': 3.557,
 'eval_steps_per_second': 0.474,
 'epoch': 1.0}

In [68]:
from transformers import pipeline
import torch

# create pipeline for sentiment classifier with custom model and tokenizer
sentiment_classifier = pipeline(task='sentiment-analysis', model=model, tokenizer=tokenizer)

Device set to use cpu


In [69]:
# let's see how our model classifies a 1 star review
# this is from (https://ca.trustpilot.com/review/www.expedia.ca?page=5)

review = """
Hote bill error, expedia.ca fault
I used to like Expedia. Now I will certainly reconsider further bookings with them.
I have booked a hotel near Cancun and the invoice from Expedia included a tax to be charged by the hotel.
This is normal, there are various and numerous "environmental" taxes these days. However, the hotel actually
requested the amount that was 3x more than what Expedia.ca provided in the invoice. As far as I understand,
it was Expedia's fault, they calculated it incorrectly. However, Expedia has rejected my claim and even the proposal
to compensate me the difference in Expedia points! I guess, this is the warning sign for me - with Expedia,
the customer is always wrong.
"""
sentiment_classifier(review)

[{'label': '1 star', 'score': 0.49091601371765137}]

In [70]:
# let's see how our model classifies a 5 star review
# this is from (https://ca.trustpilot.com/review/www.airlineticketcentre.ca)

review = """
A Grateful Customer's Appreciation for Airline Ticket
I’ve been a loyal client of Airline Ticket for over 20 years,
and their service has consistently been outstanding.
They are incredibly reliable—whether it's booking, inquiries,
or last-minute changes, they always deliver with professionalism and care.

One of the things I value most is their commitment to personal service.
They always answer their landline promptly, and it’s always a real human on
the other end—ready to help, not just route you through automated systems.
Their customer service is truly exceptional: responsive, knowledgeable, and
 genuinely dedicated to guiding and supporting customers at any time.

I’m especially thankful to Michal, Judy, and most recently Shahir
for their continued professionalism and kindness. Their expertise and
personal touch make every interaction smooth and reassuring.

Thank you for two decades of excellence!
"""
sentiment_classifier(review)

[{'label': '5 stars', 'score': 0.4177287518978119}]

In [ ]:
# let's see how our model classifies a 5 star review
# this is from (https://ca.trustpilot.com/review/www.airlineticketcentre.ca)

review = """
A Grateful Customer's Appreciation for Airline Ticket
I’ve been a loyal client of Airline Ticket for over 20 years,
and their service has consistently been outstanding.
They are incredibly reliable—whether it's booking, inquiries,
or last-minute changes, they always deliver with professionalism and care.

One of the things I value most is their commitment to personal service.
They always answer their landline promptly, and it’s always a real human on
the other end—ready to help, not just route you through automated systems.
Their customer service is truly exceptional: responsive, knowledgeable, and
 genuinely dedicated to guiding and supporting customers at any time.

I’m especially thankful to Michal, Judy, and most recently Shahir
for their continued professionalism and kindness. Their expertise and
personal touch make every interaction smooth and reassuring.

Thank you for two decades of excellence!
"""
sentiment_classifier(review)